In [ ]:
from confluent_kafka import Producer
import requests
import json
import logging
import time 

# Set up API key and API endpoint for the Article Search API
api_key = "Your_API_KEYapi_endpoint = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

# Create a Kafka producer configuration
producer_config = {
    'bootstrap.servers': 'localhost:9092' 
}

# Configure the logging module
logging.basicConfig(filename='producer.log', level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Initialize the Kafka producer
producer = Producer(producer_config)

# Fetch data from the Article Search API and produce it to a Kafka topic
page = 0  # Initialize the page number
total_articles = 0  # Initialize a counter for the total number of articles
articles_per_run = 50  # Number of articles to fetch in each run

while total_articles < 1000:  # Continue until you have at least 1000 articles
    try:
        # Make a request to the API and parse the response
        response = requests.get(api_endpoint, params={'q': 'technology', 'api-key': api_key, 'page': page})
        data = response.json()

        # Extract the articles from the API response
        articles = data.get("response", {}).get("docs", [])

        # If no articles are returned, you've reached the end of the results
        if not articles:
            break

        # Produce each article to the Kafka topic
        for article in articles:
            # Extract the fields you need
            headline = article.get("headline", {}).get("main", "No headline available")
            content = article.get("snippet", "No content available")
            news_desk = article.get("news_desk", "Unknown")
            section = article.get("section_name", "Unknown")
            source = article.get("source", "Unknown")
            word_count = article.get("word_count", 0)

            # Create a dictionary to represent the article data
            article_data = {
                "headline": headline,
                "content": content,
                "news_desk": news_desk,
                "section": section,
                "source": source,
                "word_count": word_count
            }

            # Convert the article data to JSON
            article_json = json.dumps(article_data)
            
            print("Headline:", headline)
            # Produce the article to the Kafka topic
            producer.produce('nyt', value=article_json)

            total_articles += 1  # Increment the total articles counter

        # Flush the producer to ensure data is sent to Kafka
        producer.flush()

        # Increment the page number to fetch the next page of results
        page += 1

        if total_articles % articles_per_run == 0:
            # Sleep for a while to avoid overloading the API
            time.sleep(60)

    except Exception as e:
        print(f"Error: {str(e)}")


Headline: Casey Goes to the White House + The Copyright Battle Over Artificial Intelligence + HatGPT
Headline: A Giant Leap for the Leap Second. Is Humankind Ready?
Headline: Electric Planes, Once a Fantasy, Start to Take to the Skies
Headline: The People v. Meta, Marques Brownlee on YouTube and Future Tech, and DALL-E 3 Arrives
Headline: The Race to Save Our Secrets From the Computers of the Future
Headline: Peering Into A.I.’s Black Box, Who’s the Real Techno-Optimist? and Reading Ancient Scrolls With A.I.
Headline: ‘A.I. Obama’ and Fake Newscasters: How A.I. Audio Is Swarming TikTok
Headline: Sam Bankman-Fried’s Closest Friends Become His Foes in Courtroom Clash
Headline: Unity Chief Resigns After Pricing Backlash
Headline: California’s New Senator Was a Labor Leader. Why Are Unions Upset With Her?
Headline: Elon Musk Says SpaceX Could Land on Mars in 3 to 4 Years
Headline: How a Pricing Change Led to a Revolt by Unity’s Video Game Developers
Headline: China Uses ‘Deceptive’ Methods

%6|1699488545.583|FAIL|rdkafka#producer-3| [thrd:crc-dot1x-nat-10-239-152-242.bu.edu:9092/0]: crc-dot1x-nat-10-239-152-242.bu.edu:9092/0: Disconnected (after 84732ms in state UP)
%6|1699488545.583|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Disconnected (after 86748ms in state UP)
%3|1699488545.583|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1699488545.584|FAIL|rdkafka#producer-3| [thrd:crc-dot1x-nat-10-239-152-242.bu.edu:9092/0]: crc-dot1x-nat-10-239-152-242.bu.edu:9092/0: Connect to ipv4#10.239.152.242:9092 failed: Connection refused (after 1ms in state CONNECT)
%3|1699488546.061|FAIL|rdkafka#producer-3| [thrd:crc-dot1x-nat-10-239-152-242.bu.edu:9092/0]: crc-dot1x-nat-10-239-152-242.bu.edu:9092/0: Connect to ipv4#10.239.152.242:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppress